In [ ]:
import osmnx as ox

def get_osm_network_from_point(center_point=(37.10425, 127.2335), dist=80000):
    G = ox.graph_from_point(center_point, dist=dist, network_type="drive")
    print(f"Graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    return G


network = get_osm_network_from_point()
ox.plot_graph(network)


In [ ]:
print(f"Number of nodes: {network.number_of_nodes()}")
print(f"Number of edges: {network.number_of_edges()}")

In [ ]:
# 노드, 엣지 데이터 추출
nodes, edges = ox.graph_to_gdfs(network)

In [ ]:
# 노드 데이터를 확인하시려면 주석을 제거하세요
nodes.head(1000)

In [ ]:
# 엣지 데이터를 확인하시려면 주석을 제거하세요
edges.head(2068)

In [ ]:
# 중복 노드 검증
def check_duplicate_nodes(nodes):
    # 'x', 'y' 좌표 기준 중복 확인
    duplicate_nodes = nodes.duplicated(subset=['x', 'y']).sum()
    if duplicate_nodes > 0:
        print(f"Found {duplicate_nodes} duplicate nodes based on coordinates (x, y).")
    else:
        print("No duplicate nodes found based on coordinates (x, y).")

# 중복 엣지 검증
def check_duplicate_edges(edges):
    # 'geometry'와 'length'를 기준으로 중복 확인
    duplicate_edges = edges.duplicated(subset=['geometry', 'length']).sum()
    if duplicate_edges > 0:
        print(f"Found {duplicate_edges} duplicate edges based on 'geometry' and 'length'.")
    else:
        print("No duplicate edges found based on 'geometry' and 'length'.")

# 중복 노드 검증 실행
check_duplicate_nodes(nodes)
# 중복 엣지 검증 실행
check_duplicate_edges(edges)

In [ ]:
# 출발지와 도착지 좌표
start_coords = (36.632273, 127.453050)  # 출발지: 충북대학교 정문
end_coords = (37.575182, 126.977148)    # 도착지: 서울 광화문 삼거리


In [ ]:

# 그래프에서 가장 가까운 노드 찾기
start_node = ox.distance.nearest_nodes(network, start_coords[1], start_coords[0])  # (경도, 위도)
end_node = ox.distance.nearest_nodes(network, end_coords[1], end_coords[0])

print(f"Start Node ID: {start_node}, End Node ID: {end_node}")

In [ ]:
import heapq

def dijkstra_algorithm(graph, start_node, end_node):
    # 최단 거리 테이블 초기화
    distances = {node: float('inf') for node in graph.nodes}
    predecessors = {node: None for node in graph.nodes}  # 경로 추적을 위한 이전 노드 저장
    distances[start_node] = 0

    # 우선순위 큐 (힙 구조)
    priority_queue = [(0, start_node)]  # (현재 거리, 노드)

    while priority_queue:
        # 우선순위 큐에서 가장 짧은 거리의 노드 추출
        current_distance, current_node = heapq.heappop(priority_queue)

        # 이미 처리된 노드라면 스킵
        if current_distance > distances[current_node]:
            continue

        # 인접 노드 탐색
        for neighbor, edge_data in graph[current_node].items():
            # 엣지의 길이를 가중치로 사용
            weight = edge_data[0]['length']  # 그래프 데이터에서 'length' 값
            new_distance = current_distance + weight

            # 더 짧은 경로를 발견하면 거리 갱신
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                predecessors[neighbor] = current_node
                heapq.heappush(priority_queue, (new_distance, neighbor))

    # 최단 경로 추적
    path = []
    current_node = end_node
    while current_node is not None:
        path.append(current_node)
        current_node = predecessors[current_node]

    return path[::-1], distances[end_node]  # 최단 경로와 해당 경로의 거리 반환


In [ ]:
import time

# 그래프에서 가장 가까운 노드 찾기
start_node = ox.distance.nearest_nodes(network, start_coords[1], start_coords[0])  # (경도, 위도)
end_node = ox.distance.nearest_nodes(network, end_coords[1], end_coords[0])

# 다익스트라 실행 시간 측정
start_time = time.time()
# 다익스트라 알고리즘 실행
dijkstra_path, dijkstra_distance = dijkstra_algorithm(network, start_node, end_node)
dijkstra_time = time.time() - start_time

print("Shortest Path:", dijkstra_path)
print("Total Distance (m):", dijkstra_distance)
print(f"Dijkstra Execution Time: {dijkstra_time:.4f} seconds")
ox.plot_graph_route(network, route=dijkstra_path, route_linewidth=3)

In [ ]:
def bellman_ford_algorithm(graph, start_node, end_node):
    # 최단 거리 테이블 초기화
    distances = {node: float('inf') for node in graph.nodes}
    predecessors = {node: None for node in graph.nodes}  # 경로 추적을 위한 이전 노드 저장
    distances[start_node] = 0  # 출발 노드의 최단 거리는 0

    # 노드 개수
    num_nodes = len(graph.nodes)

    # 모든 엣지에 대해 (노드 개수 - 1)번 반복
    for _ in range(num_nodes - 1):
        for u, v, edge_data in graph.edges(data=True):
            weight = edge_data['length']  # 엣지 가중치
            if distances[u] + weight < distances[v]:
                distances[v] = distances[u] + weight
                predecessors[v] = u

    # 음수 사이클 확인 (선택 사항)
    for u, v, edge_data in graph.edges(data=True):
        weight = edge_data['length']
        if distances[u] + weight < distances[v]:
            raise ValueError("Negative weight cycle detected")

    # 최단 경로 추적
    path = []
    current_node = end_node
    while current_node is not None:
        path.append(current_node)
        current_node = predecessors[current_node]

    return path[::-1], distances[end_node]  # 최단 경로와 해당 경로의 거리 반환


In [ ]:
# 그래프에서 가장 가까운 노드 찾기
start_node = ox.distance.nearest_nodes(network, start_coords[1], start_coords[0])  # (경도, 위도)
end_node = ox.distance.nearest_nodes(network, end_coords[1], end_coords[0])

# 벨만-포드 실행 시간 측정
start_time = time.time()
bellman_ford_path, bellman_ford_distance = bellman_ford_algorithm(network, start_node, end_node)
bellman_ford_time = time.time() - start_time

# 결과 출력
print("Shortest Path (Bellman-Ford):", bellman_ford_path)
print("Total Distance (m):", bellman_ford_distance)
print(f"Bellman-Ford Execution Time: {bellman_ford_time:.4f} seconds")

# 경로 시각화
ox.plot_graph_route(network, route=bellman_ford_path, route_linewidth=3)

In [ ]:
# 경로 비교
if dijkstra_path == bellman_ford_path:
    print("The paths are identical.")
else:
    print("The paths are different.")
    print("Dijkstra Path:", dijkstra_path)
    print("Bellman-Ford Path:", bellman_ford_path)

# 총 거리 비교
if dijkstra_distance == bellman_ford_distance:
    print("The total distances are identical.")
else:
    print("The total distances are different.")
    print(f"Dijkstra Distance: {dijkstra_distance} meters")
    print(f"Bellman-Ford Distance: {bellman_ford_distance} meters")

# 두 경로를 시각적으로 비교
ox.plot_graph_routes(
    network,
    routes=[dijkstra_path, bellman_ford_path],
    route_colors=['red', 'blue'],  # 다익스트라는 빨간색, 벨만-포드는 파란색
    route_linewidth=3,
)



In [ ]:
import heapq
import time

# 단순 리스트 기반 다익스트라
def dijkstra_with_list(graph, start_node, end_node):
    distances = {node: float('inf') for node in graph.nodes}
    predecessors = {node: None for node in graph.nodes}
    distances[start_node] = 0

    # 리스트 사용
    unvisited_nodes = list(graph.nodes)
    while unvisited_nodes:
        # 리스트에서 최소 거리 노드 선택
        current_node = min(unvisited_nodes, key=lambda node: distances[node])
        unvisited_nodes.remove(current_node)

        if distances[current_node] == float('inf'):
            break

        for neighbor, edge_data in graph[current_node].items():
            weight = edge_data[0]['length']
            new_distance = distances[current_node] + weight
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                predecessors[neighbor] = current_node

    # 최단 경로 추적
    path = []
    current_node = end_node
    while current_node is not None:
        path.append(current_node)
        current_node = predecessors[current_node]

    return path[::-1], distances[end_node]

# 테스트 실행
start_time = time.time()
_, list_distance = dijkstra_with_list(network, start_node, end_node)
list_time = time.time() - start_time

start_time = time.time()
_, heap_distance = dijkstra_algorithm(network, start_node, end_node)
heap_time = time.time() - start_time

print(f"List-Based Dijkstra: Distance = {list_distance}, Time = {list_time:.6f} seconds")
print(f"Heap-Based Dijkstra: Distance = {heap_distance}, Time = {heap_time:.6f} seconds")
